In [1]:
from bs4 import BeautifulSoup
import requests
from lxml import etree
import csv
import time

headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }

In [2]:
def save_rapp_giuridico(tab, writer, id_):
    name = tab[0].text
    name = name.replace('\n', '')
    code = tab[1].text
    code = code.replace('\n', '')
    role = tab[2].text
    role = role.replace('\n', '')
    start = tab[3].text
    start = start.replace('\n', '')
    row = [id_, name, code, role, start]
    #print(row)
    writer.writerow(row)
    
    
def save_partner(id_, tab, writer):
    for i in tab:
        figli = i.getchildren()
        part = figli[0].text
        part = part.replace('\n', '')
        contr = figli[1].text
        contr = contr.replace('\n', '')
        name = figli[2].text
        name = name.replace('\n', '')
        code = figli[3].text
        code = code.replace('\n', '')
        row = [id_, name, code, part, contr]
        writer.writerow(row)
        
def save_activity(id_, act, writer):
    area = act[1].text
    EMTAK = act[3].text
    NACE = act[5].text
    source = act[7].text
    source = source.replace('\n', '')
    row = (id_, area, EMTAK, NACE, source)
    writer.writerow(row)

In [7]:
def get_info(url, id_, writer, writer_rapp, writer_part, writer_act):
    pre_req = time.time_ns()
    req = requests.get(url, headers)
    post_req = time.time_ns()
    tot_req = post_req - pre_req
    soup = BeautifulSoup(req.content, 'html.parser')
    dom = etree.HTML(str(soup))
    
    #Company name
    pre_name = time.time_ns()
    name = dom.xpath('//div[@class = "h2 text-primary mb-2"]')
    post_name = time.time_ns()
    tot_name = post_name - pre_name
    if name:
        name_txt = name[0].text
        name_txt = name_txt.replace(' ', '')
        name_txt = name_txt.replace('\n', '')
        print(name_txt)
    else:
        name_txt = None
        
    #Registry code
    pre_code = time.time_ns()
    code = dom.xpath('//div[text() = "Registry code"]/../div[2]')
    post_code = time.time_ns()
    tot_code = post_code - pre_code
    if code:
        code_txt = code[0].text
    else:
        code_txt = None
    
    #Legal form
    pre_lf = time.time_ns()
    lf = dom.xpath('//div[text() = "Legal form"]/../div[2]')
    post_lf = time.time_ns()
    tot_lf = post_lf - pre_lf
    if lf:
        lf_txt = lf[0].text
    else:
        lf_txt = None
    
    #Status
    pre_del = time.time_ns()
    deleted = dom.xpath('//div[text() = "Deleted"]/../div[2]')
    post_del = time.time_ns()
    tot_del = post_del - pre_del
    if deleted: 
        status_txt = 'Deleted'
        deleted_date = deleted[0].text
        capital_txt = None
        add_txt =None
        tot_add = None
        tot_status = None
        tot_cap = None
    else:
        pre_status = time.time_ns()
        status = dom.xpath('//div[text() = "Status"]/../div[2]/text()')
        post_status = time.time_ns()
        tot_status = post_status - pre_status
        deleted_date = None
        if status:
            status_txt = status[1]
            status_txt = status_txt.replace('\n', '')
            status_txt = status_txt.replace(' ', '')
        else: 
            status_txt = None
        #Capital
        pre_cap = time.time_ns()
        capital = dom.xpath('//div[text() = "Capital"]/../div[2]')
        post_cap = time.time_ns()
        tot_cap = post_cap - pre_cap
        if capital:
            capital_txt = capital[0].text
            capital_txt = capital_txt.replace(' ', '')
            capital_txt = capital_txt.replace('\n', '')
        else:
            capital_txt = None
            
        #Address
        pre_add = time.time_ns()
        address = dom.xpath('//div[text() = "Address"]/../div[2]')
        post_add = time.time_ns()
        tot_add = post_add - pre_add
        if address:
            add_txt = address[0].text
            add_txt = add_txt.replace('\n', '')
            add_txt = add_txt.replace(',', '')
        else:
            add_txt = None
    
    #Rappresentante giuridico
    pre_rapp = time.time_ns()
    tab = dom.xpath('//div[contains(text(), "Right of representation")]/../div[2]/table/tbody/tr/td')
    post_rapp = time.time_ns()
    tot_rapp = post_rapp - pre_rapp
    if tab:
        save_rapp_giuridico(tab, writer_rapp, id_)
    
    #Registered
    pre_reg = time.time_ns()
    registered = dom.xpath('//div[text() = "Registered"]/../div[2]')
    post_reg = time.time_ns()
    tot_reg = post_reg - pre_reg
    if registered:
        registered_txt = registered[0].text
    else:
        registered_txt = None
    
    #Principal activity
    pre_act = time.time_ns()
    activity = dom.xpath('//div[contains(text(), "activity")]/../div/table/tbody/tr[2]/td/div/div')
    post_act = time.time_ns()
    tot_act = post_act - pre_act
    if activity:
        save_activity(id_, activity, writer_act)
    
    #Partners
    pre_part = time.time_ns()
    part = dom.xpath('//div[contains(text(), "Partners")]/../../div[3]/table/tbody/tr')
    post_part = time.time_ns()
    tot_part = post_part - pre_part
    if part:
        save_partner(id_, part, writer_part)
    
    row = [id_, url, name_txt, code_txt, lf_txt, status_txt, registered_txt, capital_txt, add_txt, deleted_date, tot_req, 
          tot_name, tot_code, tot_lf, tot_status, tot_reg, tot_cap, tot_add, tot_del, tot_rapp, tot_part, tot_act]
    writer.writerow(row)

In [9]:
id_ = 0
url = 'https://ariregister.rik.ee'

head = ['ID', 'URL', 'Name', 'Code', 'Legal form', 'Status', 'Registration Date', 'Capital', 'Address', 'Deletion Date','Tempo richiesta',
        'Tempo estrazione Name', 'Tempo estrazione Code', 'Tempo estrazione Legal Form', 'Tempo estrazione Status', 
        'Tempo estrazione Reg. Date', 'Tempo estrazione Capital', 'Tempo estrazione Address', 'Tempo estrazione Del. Date', 'Tempo estrazione dati Rapp.',
        'Tempo estrazione Partners', 'Tempo estrazione Activity']

ds = open('ebusiness-aziende-dataset.csv', 'w')
writer = csv.writer(ds)
writer.writerow(head)

head_r = ['ID azienda', 'Name', 'Code', 'Role', 'Start Date']

ds_r = open('ebusiness-rapp-dataset.csv', 'w')
writer_r = csv.writer(ds_r)
writer_r.writerow(head_r)

head_p = ['ID azienda', 'Name', 'Code', 'Participation', 'Contribution']

ds_p = open('ebusiness-partners-dataset.csv', 'w')
writer_p = csv.writer(ds_p)
writer_p.writerow(head_p)

head_a = ['ID azienda', 'Area of Activity', 'EMTAK Code', 'NACE Code', 'Source']

ds_a = open('ebusiness-activity-dataset.csv', 'w')
writer_a = csv.writer(ds_a)
writer_a.writerow(head_a)

d = {}
start = time.time_ns()
for x in range(1, 75):  #(1, 75)
    url2 = url + '/eng/company_search_result/56880fa?name_or_code=spa&page=' + str(x)
    start_req = time.time_ns()
    req = requests.get(url2, headers)
    end_req = time.time_ns()
    tot_req = end_req - start_req
    start_est = time.time_ns()
    soup = BeautifulSoup(req.content, 'html.parser')
    dom = etree.HTML(str(soup))
    links = dom.xpath('//td/div/a/@href')
    end_est = time.time_ns()
    tot_est = end_est - start_est
    d[str(x)] = [tot_req, tot_est]
    print(x)
    for link in links:
        id_ += 1
        get_info(url + link, id_, writer, writer_r, writer_p, writer_a)
        time.sleep(2)

ds.close()
ds_r.close()
ds_p.close()
ds_a.close()
end = time.time_ns()
print(end - start, 'nanosecondi')
print(id_, 'aziende')
#print(d)

1
\n10DspaceO\xc3\x9c\n(14318875)\n
\n21SparrowsO\xc3\x9c\n(16270291)\n\n
\n3DSTO\xc3\x9c\n(12454634)\n\n
\n8BitsSpaceO\xc3\x9c\n(14867109)\n\n
\nA1EhituspartnerO\xc3\x9c\n(16451101)\n\n
\nosa\xc3\xbchingAarspak\n(10166769)\n\n
\nAarspakTootmineO\xc3\x9c\n(11959421)\n\n
\nA&ASpamarketO\xc3\x9c\n(11945353)\n
\nO\xc3\x9cABCOnlineSolution\n(16008334)\n\n
\nO\xc3\x9cABCRakennuspalvelut\n(11739760)\n
\nAbovespaceO\xc3\x9c\n(16322007)\n\n
\nAbrukaSPAO\xc3\x9c\n(16042743)\n\n
\nABuildingGroupO\xc3\x9c\n(14166276)\n
\nAceofSpaceO\xc3\x9c\n(12613707)\n\n
\nAdmiralFerdinandvonWrangelliM\xc3\xa4lestuspaikadeHooldamisejaJ\xc3\xa4\xc3\xa4dvustamiseOsa\xc3\xbching\n(10337568)\n
\nADUSPAGROUPO\xc3\x9c\n(12561784)\n
\nAdvokaadib\xc3\xbcrooLionLawO\xc3\x9c\n(11515492)\n\n
\nAerospaceQualityO\xc3\x9c\n(14568307)\n\n
\nAespaangaarO\xc3\x9c\n(12366410)\n\n
\nOsa\xc3\xbchingAespaEhitus\n(11373307)\n\n
2
\nAespaInvestO\xc3\x9c\n(14855410)\n\n
\nO\xc3\x9cAESPAJUUKSUR\n(12260290)\n\n
\nMittetulundus\xc3\xbchi

\nCaspartHouseO\xc3\x9c\n(12643588)\n\n
\nCaspartReedesO\xc3\x9c\n(12645280)\n\n
\nCHAINWAYSLIMITEDO\xc3\x9c\n(14928728)\n\n
\nChiekhGaspardO\xc3\x9c\n(11188969)\n
\nCircularSpaceO\xc3\x9c\n(16585528)\n\n
\nClassicRakennuspalvelutO\xc3\x9c\n(11931279)\n\n
\nCNMedcapitalO\xc3\x9c\n(16088280)\n\n
\nCoffeeClubO\xc3\x9c\n(11587089)\n\n
11
\nCoinspackO\xc3\x9c\n(14675888)\n\n
\nCopterCapitalO\xc3\x9c\n(11325679)\n\n
\nCORALSPAO\xc3\x9c\n(11727610)\n\n
\nMittetulundus\xc3\xbchingCoworkingSpaceTallinn\n(80340595)\n
\nCoworkSpaceO\xc3\x9c\n(14784063)\n\n
\nCreativeSpace(Europe)O\xc3\x9c\n(14860812)\n\n
\nCreativeSpaceStudio\n(80602457)\n\n
\nCrowdspaceO\xc3\x9c\n(16536518)\n\n
\nCryptoBaySpaceO\xc3\x9c\n(16354705)\n\n
\nCryptoXpressO\xc3\x9c\n(14845698)\n\n
\nCrystalSolutionsO\xc3\x9c\n(12084841)\n\n
\nCrystalspaceO\xc3\x9c\n(12564073)\n\n
\nCSpaceHostingsO\xc3\x9c\n(14730392)\n\n
\nCutinelliSpartimentoO\xc3\x9c\n(11339753)\n\n
\nCyberWiseSpaceO\xc3\x9c\n(14999935)\n\n
\nDAOSpaceO\xc3\x9c\n(16

\nGoToSpaO\xc3\x9c\n(14448170)\n\n
\nGRAFFSPARKO\xc3\x9c\n(14344571)\n\n
\nGrandRoseSpaO\xc3\x9c\n(11302098)\n
\nGravitySpaceO\xc3\x9c\n(16526690)\n\n
\nO\xc3\x9cGreenspace\n(11666951)\n\n
\nGreetingsfromKasparO\xc3\x9c\n(11150085)\n\n
\nGreySpaceO\xc3\x9c\n(16481846)\n\n
\ngrowthspartanO\xc3\x9c\n(12968037)\n
\nGSPALVELUTO\xc3\x9c\n(14099798)\n
\nGVFinanceO\xc3\x9c\n(12059263)\n\n
\nO\xc3\x9cHAAElektrit\xc3\xb6\xc3\xb6d\n(11920465)\n\n
\nHaanjaLooduspargiAdministratsioon\n(70001722)\n
21
\nHaanjalooduspargin\xc3\xb5ukoda\n(80087328)\n
\nHaapsaluKlaaspakettO\xc3\x9c\n(10995478)\n
\nHaapsaluSeiklusparkO\xc3\x9c\n(14814491)\n\n
\nHALDUSPARTNERO\xc3\x9c\n(14445869)\n\n
\nHalduspartnerO\xc3\x9c\n(11124662)\n
\nHaljastuspartneridO\xc3\x9c\n(11347406)\n\n
\nHamadaGlobalBusinessPartnersO\xc3\x9c\n(14980396)\n\n
\nOsa\xc3\xbchingHanspak\n(10707158)\n
\nHarryKaunispaik\n(10761220)\n
\nMT\xc3\x9cHCSPARTATALLINN\n(80555969)\n\n
\nHelgiKappIstumispadjad\n(11855868)\n
\nHeliaedO\xc3\x9c\n(11341187)

\nKindlustuspartnerOsa\xc3\xbching\n(10954976)\n\n
\nKingspanInsulationO\xc3\x9c\n(12389210)\n\n
\nKingspanO\xc3\x9c\n(11459813)\n\n
\nKinnisparkO\xc3\x9c\n(16385746)\n\n
\nKirnaLoodusravipark\n(80235154)\n
\nO\xc3\x9cKivikaevur\n(11339463)\n
\naktsiaseltsKlaaspakett\n(10048686)\n
31
\nKMGMTSpaceO\xc3\x9c\n(14243775)\n\n
\nKMIGruppO\xc3\x9c\n(12681732)\n\n
\nKoduspaO\xc3\x9c\n(11958031)\n\n
\nKoeraSPAO\xc3\x9c\n(14747004)\n\n
\nKohilaSpaceO\xc3\x9c\n(16270762)\n\n
\nKohtumispaikO\xc3\x9c\n(14478679)\n\n
\nKolimispartnerO\xc3\x9c\n(12662611)\n\n
\nKombainiGruppO\xc3\x9c\n(11476728)\n
\nKoolituspartnerO\xc3\x9c\n(11083785)\n\n
\nKoristuspartnerO\xc3\x9c\n(16460063)\n\n
\nkorvpalliklubiSPARK-S\n(80037916)\n
\nKRISPARO\xc3\x9c\n(11818927)\n\n
\nKruonisO\xc3\x9c\n(10630713)\n
\nK-SPACEMT\xc3\x9c\n(80397632)\n\n
\nO\xc3\x9cKSSpark\n(16208903)\n\n
\nKuressaareSeiklusparkO\xc3\x9c\n(16225103)\n\n
\nKuusaluT\xc3\xb6\xc3\xb6stusparkO\xc3\x9c\n(11663100)\n
\nmittetulundus\xc3\xbchingKuusikuP\xc3\

\nPAKRITeadus-jaT\xc3\xb6\xc3\xb6stusparkO\xc3\x9c\n(12529763)\n\n
\nPalermoCateringO\xc3\x9c\n(10957093)\n
\nPangaimotuHolongaO\xc3\x9c\n(14329654)\n\n
\nPaspartuutransO\xc3\x9c\n(16327080)\n\n
\nO\xc3\x9cPeipsiEhitus\n(11157182)\n
\nMT\xc3\x9cPereLooduspaik\n(80606863)\n\n
\nPesuSPAO\xc3\x9c\n(14136967)\n\n
\nOsa\xc3\xbchingPhotoFactory\n(11517158)\n
\nOsa\xc3\xbchingPispar\n(10073342)\n\n
\nPlanetspaceO\xc3\x9c\n(16456203)\n\n
\nPLSpaceO\xc3\x9c\n(12564587)\n\n
\nPluspakO\xc3\x9c\n(10543428)\n\n
\nOsa\xc3\xbchingPLUSSPANUS\n(10611785)\n\n
41
\nosa\xc3\xbchingPlusspartner\n(10487813)\n\n
\nPodSpaceO\xc3\x9c\n(14339722)\n\n
\nPohjois-suomenRakennuspalveluO\xc3\x9c\n(11224116)\n
\nPOLARSPACEO\xc3\x9c\n(11204444)\n\n
\nPolyverseO\xc3\x9c\n(14212697)\n\n
\nPostimajaDaySpaO\xc3\x9c\n(14947192)\n\n
\nPressmy.spaceO\xc3\x9c\n(16411662)\n\n
\nPrivilegDaySpaO\xc3\x9c\n(12088230)\n\n
\nProEhituspartnerO\xc3\x9c\n(14057245)\n\n
\nProgressPartnerO\xc3\x9c\n(11399614)\n\n
\nO\xc3\x9cProspace\n(11

\nSpaceLineO\xc3\x9c\n(12957737)\n\n
\nOsa\xc3\xbchingSpacelord\n(11502975)\n
\nSpaceLoveO\xc3\x9c\n(14636279)\n\n
\nSpacemanO\xc3\x9c\n(11352699)\n\n
\nO\xc3\x9cSpacemasters\n(11591919)\n\n
\nSpaceMetO\xc3\x9c\n(12682004)\n
\nSpacemindEuropeO\xc3\x9c\n(14596440)\n\n
\nO\xc3\x9cSPACEMONKEY\n(16503720)\n\n
\nSpaceportO\xc3\x9c\n(14614160)\n\n
\nSpaceProductionsOsa\xc3\xbching\n(11549924)\n\n
\nSpaceProjectO\xc3\x9c\n(12442461)\n\n
\nSpaceRabbitO\xc3\x9c\n(12416562)\n\n
\nSpacerepO\xc3\x9c\n(16593350)\n\n
\nSpacerHBO\xc3\x9c\n(14712856)\n\n
\nSpaceRIderO\xc3\x9c\n(11694997)\n
\nSpaceRunO\xc3\x9c\n(11322617)\n
\nSPACERWORLDO\xc3\x9c\n(14743549)\n\n
51
\nSpaceSevenO\xc3\x9c\n(14971090)\n\n
\nO\xc3\x9cSpaceshark\n(12852636)\n\n
\nSpaceshipO\xc3\x9c\n(12252190)\n
\nSpaceShuttlO\xc3\x9c\n(14897979)\n\n
\nspacesideO\xc3\x9c\n(16134731)\n\n
\nSpaceSoftO\xc3\x9c\n(16610663)\n\n
\nSpaceSolutionBalticO\xc3\x9c\n(11646478)\n
\nOsa\xc3\xbchingSpacestar\n(11309114)\n\n
\nSPACESYSO\xc3\x9c\n(11135174)

\nSparkTimeO\xc3\x9c\n(12448065)\n\n
\nSparkTrioO\xc3\x9c\n(10418829)\n
\nSparkupO\xc3\x9c\n(16246588)\n\n
\nSparkWeldO\xc3\x9c\n(16566653)\n\n
\nOsa\xc3\xbchingSparkwell\n(11018977)\n
\nSparkyO\xc3\x9c\n(16311015)\n\n
\nO\xc3\x9cSPARKY\n(11005688)\n
\nSparkySpotO\xc3\x9c\n(14865560)\n\n
\nO\xc3\x9cSparoberg\n(10957207)\n\n
\nSparofO\xc3\x9c\n(12087012)\n
\nO\xc3\x9cSparohansa\n(10956797)\n\n
61
\nOsa\xc3\xbchingSparolink\n(10957087)\n\n
\nosa\xc3\xbchingSPARossia\n(11168688)\n\n
\nSparotecO\xc3\x9c\n(12197054)\n\n
\nSparqO\xc3\x9c\n(14721602)\n\n
\nSparrekArtO\xc3\x9c\n(12639109)\n\n
\nosa\xc3\xbchingSparren\n(10931314)\n
\nSparringO\xc3\x9c\n(12308619)\n\n
\nOsa\xc3\xbchingSPARROW\n(10371051)\n
\nSparrowSolutionsO\xc3\x9c\n(12973096)\n\n
\nsparse.techO\xc3\x9c\n(14150884)\n\n
\nSparshInvestmentsO\xc3\x9c\n(14533289)\n\n
\nOsa\xc3\xbchingSpars\n(10260736)\n\n
\nSpartaAkrobaatikaklubi\n(80292011)\n\n
\nSpartaCapitalO\xc3\x9c\n(12974256)\n\n
\nSpartaCarO\xc3\x9c\n(10803875)\n
\nSPARTACU

\nValguspartnerO\xc3\x9c\n(14483545)\n
\nValueSpaceO\xc3\x9c\n(16481438)\n\n
\nVaramiespalveluO\xc3\x9c\n(12711446)\n
\nVarepu-Kaspar\n(10596206)\n\n
\nOsa\xc3\xbchingVASPAVEL\n(10347518)\n
\nVassiliBESPAL\xc3\x95I\n(10814873)\n\n
\nO\xc3\x9cVEDESPAS\n(11645697)\n
\nVehklemisklubiESPADA\n(80140287)\n
\nVelevstarO\xc3\x9c\n(10956834)\n
\nVELLGRENAO\xc3\x9c\n(12075523)\n
\nVENE\xc3\x9cHTSUSPARTEI\n(80035685)\n
\nVenturaInvesteerimiseAktsiaselts\n(10179773)\n
\nVeroLogisticsO\xc3\x9c\n(11256889)\n\n
\nAktsiaseltsVESPALO\n(10364105)\n\n
\nOsa\xc3\xbchingVESPAL\n(10212708)\n
71
\nVespanaO\xc3\x9c\n(16427077)\n\n
\nVESPARTSO\xc3\x9c\n(16477847)\n\n
\nO\xc3\x9cVestmanT\xc3\xb6\xc3\xb6stuspargid\n(11364490)\n
\nO\xc3\x9cVideoExpert\n(11191428)\n\n
\nOsa\xc3\xbchingViikingEhituspartner\n(11309947)\n\n
\nViimistluspartnerO\xc3\x9c\n(12483386)\n\n
\nViimsiSeiklusparkO\xc3\x9c\n(12460238)\n\n
\nO\xc3\x9cViimsiSPA\n(11298913)\n
\nViimsiT\xc3\xb6\xc3\xb6stusparkO\xc3\x9c\n(10919313)\n\n
\nViktorMasp

In [10]:
print(d)

{'1': [1981856600, 232307800], '2': [383812100, 59600200], '3': [407320500, 57937500], '4': [322714600, 64555000], '5': [274603600, 58591900], '6': [584368500, 51336000], '7': [566317600, 62037400], '8': [432814400, 60559200], '9': [620163600, 55010900], '10': [509208900, 127586500], '11': [962426300, 57649500], '12': [782140400, 62559600], '13': [674766500, 61600300], '14': [703594300, 58564900], '15': [472432000, 58965400], '16': [674452100, 55637900], '17': [752674100, 51553200], '18': [334678700, 50055000], '19': [625990800, 61644200], '20': [306303200, 52633700], '21': [1348246400, 60554100], '22': [531079100, 82927400], '23': [956839700, 51497000], '24': [756999700, 53571900], '25': [626255100, 52593400], '26': [535769000, 60006800], '27': [290221100, 62549300], '28': [765983700, 50166200], '29': [402957200, 62632100], '30': [371955100, 57607600], '31': [293691700, 60615800], '32': [800203600, 62631300], '33': [303272000, 59047600], '34': [362801700, 50960800], '35': [899314400, 